In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = 500

%matplotlib inline

In [37]:
df_users = pd.read_csv(r'C:\Users\Admin\Downloads\1481069814_relax_challenge\relax_challenge\takehome_users.csv',encoding='latin-1',  index_col=0)
df_users

,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
object_id,,,,,,,,,
1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0
...,...,...,...,...,...,...,...,...,...
11996,2013-09-06 06:14:15,Meier Sophia,SophiaMeier@gustr.com,ORG_INVITE,1.378448e+09,0,0,89,8263.0
11997,2013-01-10 18:28:37,Fisher Amelie,AmelieFisher@gmail.com,SIGNUP_GOOGLE_AUTH,1.358275e+09,0,0,200,NaN
11998,2014-04-27 12:45:16,Haynes Jake,JakeHaynes@cuvox.de,GUEST_INVITE,1.398603e+09,1,1,83,8074.0


In [35]:
df_engage = pd.read_csv(r'C:\Users\Admin\Downloads\1481069814_relax_challenge\relax_challenge\takehome_user_engagement.csv')
df_engage

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1
...,...,...,...
207912,2013-09-06 06:14:15,11996,1
207913,2013-01-15 18:28:37,11997,1
207914,2014-04-27 12:45:16,11998,1
207915,2012-06-02 11:55:59,11999,1


# Building the "Adopted User" Label

First, I will build the "adopted user" label from the user engagement dataset, which I have loaded into the df_engage dataframe as shown above. Let's confirm that there's no missing data in that dataset:

In [4]:
# checking for any missing data
df_engage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   time_stamp  207917 non-null  object
 1   user_id     207917 non-null  int64 
 2   visited     207917 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.8+ MB


It looks like we have a dataset of 207,917 rows of logins into the system with no missing values. The 'time_stamp' column provides the date and time for each login, but was loaded in as strings in the dataframe. I will next convert that data into a datetime format and set it as the index for the dataframe:

In [5]:
# convert from string to datetime format
df_engage['time_stamp'] = pd.to_datetime(df_engage['time_stamp'])

In [6]:
# set the time_stamp column as the dataframe index
df_engage = df_engage.set_index('time_stamp')
df_engage

,user_id,visited
time_stamp,,
2014-04-22 03:53:30,1,1
2013-11-15 03:45:04,2,1
2013-11-29 03:45:04,2,1
2013-12-09 03:45:04,2,1
2013-12-25 03:45:04,2,1
...,...,...
2013-09-06 06:14:15,11996,1
2013-01-15 18:28:37,11997,1
2014-04-27 12:45:16,11998,1


Now that we have a datetime index for the dataframe, I can use the .resample() method to convert the dates into weeks. While this won't perfectly capture every 7-day period in which a user may have adopted the product, it should capture the lion's share of the 3-times-a-week users and therefore should suffice for the purposes of this cursory analysis. I will group the data by 'user_id' first, then resample the dates, and sum the data so that the 'visited' column will show us which users are 'adopted':

In [7]:
# group data by user_id, resample into weekly dates, and sum the visited column
df_engage = df_engage.groupby('user_id').resample('1W').sum()
df_engage

user_id  visited
user_id time_stamp                  
1       2014-04-27        1        1
2       2013-11-17        2        1
        2013-11-24        0        0
        2013-12-01        2        1
        2013-12-08        0        0
...                     ...      ...
11996   2013-09-08    11996        1
11997   2013-01-20    11997        1
11998   2014-04-27    11998        1
11999   2012-06-03    11999        1
12000   2014-01-26    12000        1

[82282 rows x 2 columns]

Next, I will isolate the particular users that have logged into the product at least three times in a week:

In [8]:
# create dataframe of only the adopted users
df_adopted = df_engage[df_engage['visited'] > 2]
del df_adopted['user_id']
df_adopted

visited
user_id time_stamp         
2       2014-02-09        3
10      2013-03-03        3
        2013-04-14        3
        2013-04-28        3
        2013-05-05        4
...                     ...
11988   2014-04-06        3
        2014-05-04        4
        2014-05-18        3
        2014-05-25        5
        2014-06-01        3

[33859 rows x 1 columns]

We have 33,859 weekly datapoints of over 3 logins to the product in a week, but not every one of these is a unique user. I therefore will create a list of adopted user ids to use to create the label for the other dataset:

In [9]:
# create user id list for adopted users
adopted_index = list(df_adopted.index)
adopted_users_list = []
for i in range(len(adopted_index)):
    adopted_users_list.append(adopted_index[i][0])
len(adopted_users_list)

33859

In [10]:
# remove duplicate user ids
adopted_users = set(adopted_users_list)
len(adopted_users)

1445

We have calculated that 1,445 of the 12,000 users have adopted the product, which makes up a modest 12% of the total users. Let's add the adopted label to the df_users dataframe:

In [11]:
# add an adopted column to the dataframe with a default value of False
df_users['adopted'] = False

# use the adopted_users set to label the adopted users in the dataframe
for user in adopted_users:
    df_users.loc[user, 'adopted'] = True

df_users.head(10)

,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted
object_id,,,,,,,,,,
1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,False
2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,True
3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,False
4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,False
5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,False
6,2013-12-17 03:37:06,Cunha Eduardo,EduardoPereiraCunha@yahoo.com,GUEST_INVITE,1.387424e+09,0,0,197,11241.0,False
7,2012-12-16 13:24:32,Sewell Tyler,TylerSewell@jourrapide.com,SIGNUP,1.356010e+09,0,1,37,NaN,False
8,2013-07-31 05:34:02,Hamilton Danielle,DanielleHamilton@yahoo.com,PERSONAL_PROJECTS,NaN,1,1,74,NaN,False
9,2013-11-05 04:04:24,Amsel Paul,PaulAmsel@hotmail.com,PERSONAL_PROJECTS,NaN,0,0,302,NaN,False


# Data Preprocessing and Exploration

With the adopted user label now added, I will now inspect the df_users dataframe to do any needed data cleaning and gain a better understanding of the dataset features

In [12]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 1 to 12000
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   creation_time               12000 non-null  object 
 1   name                        12000 non-null  object 
 2   email                       12000 non-null  object 
 3   creation_source             12000 non-null  object 
 4   last_session_creation_time  8823 non-null   float64
 5   opted_in_to_mailing_list    12000 non-null  int64  
 6   enabled_for_marketing_drip  12000 non-null  int64  
 7   org_id                      12000 non-null  int64  
 8   invited_by_user_id          6417 non-null   float64
 9   adopted                     12000 non-null  bool   
dtypes: bool(1), float64(2), int64(3), object(4)
memory usage: 1.2+ MB


It looks like two of the columns have missing data values. The 'last_session_creation_time' column is missing data for roughly a third of the users. I am going to make a judgment call to simply not use that feature in our model, as I feel it is likely to be correlated to our adopted user label (most likely the adopted users will have more recent last sessions) and therefore could limit the signficance of our model results. In other words, I do not think it would be overly insightful to build a model that tells us that the more recent a user has logged in, the more likely he or see is to have adopted the product.

As for the missing values in the 'invited_by_user_id' feature, let's investigate:

In [13]:
df_users['invited_by_user_id'].value_counts(ascending=False, dropna=False)

NaN        5583
10741.0      13
2527.0       12
1525.0       11
2308.0       11
           ... 
6690.0        1
9310.0        1
10801.0       1
5585.0        1
10872.0       1
Name: invited_by_user_id, Length: 2565, dtype: int64

It looks like the 'invited_by_user_id' feature provides the particular user_id that invited the user or, alternatively, provides a NaN value if the user was not invited by another user. I am going to convert this feature into boolean data (indicating whether the user was invited to the product by another existing user) for use in the model:

In [14]:
df_users['invited_by_user'] = df_users['invited_by_user_id'].isnull() == False
del df_users['invited_by_user_id']
df_users.head()

,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,adopted,invited_by_user
object_id,,,,,,,,,,
1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,False,True
2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,True,True
3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,False,True
4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,False,True
5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,False,True


Email addresses--
The dataset also provides the individual email adddress for each user. While each particular address likely won't be helpful for the model, it may be helpful to capture the most common email domains of the product users. Let's extract that information:

In [15]:
# split the individual email address from email domain
df_users['email_split'] = df_users['email'].str.split('@')
del df_users['email']

# extract the email domains into separate column
for user in range(len(df_users)):
    df_users.loc[user+1, 'email_domain'] = df_users.loc[user+1, 'email_split'][1]
del df_users['email_split']

# evaluate the most common email domains
df_users['email_domain'].value_counts()[:10]

gmail.com         3562
yahoo.com         2447
jourrapide.com    1259
cuvox.de          1202
gustr.com         1179
hotmail.com       1165
dqwln.com            2
oqpze.com            2
qgjbc.com            2
mumdc.com            2
Name: email_domain, dtype: int64

It looks like there are six common email domains among the users. Let's convert the uncommon domains into their own 'Other' value so that we can use this data as a categorical feature for the model:

In [16]:
# create final email column
df_users['email'] = 'other'

# populate the final email column with the common email domains
for user in range(len(df_users)):
    if df_users.loc[user+1, 'email_domain'] == 'gmail.com':
        df_users.loc[user+1, 'email'] = 'gmail.com'
    if df_users.loc[user+1, 'email_domain'] == 'yahoo.com':
        df_users.loc[user+1, 'email'] = 'yahoo.com'
    if df_users.loc[user+1, 'email_domain'] == 'jourrapide.com':
        df_users.loc[user+1, 'email'] = 'jourrapide.com'
    if df_users.loc[user+1, 'email_domain'] == 'cuvox.de':
        df_users.loc[user+1, 'email'] = 'cuvox.de'
    if df_users.loc[user+1, 'email_domain'] == 'gustr.com':
        df_users.loc[user+1, 'email'] = 'gustr.com'
    if df_users.loc[user+1, 'email_domain'] == 'hotmail.com':
        df_users.loc[user+1, 'email'] = 'hotmail.com'

del df_users['email_domain']

df_users['email'].value_counts(ascending=False)

gmail.com         3562
yahoo.com         2447
jourrapide.com    1259
cuvox.de          1202
other             1186
gustr.com         1179
hotmail.com       1165
Name: email, dtype: int64

In [17]:
df_users.head()

,creation_time,name,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,adopted,invited_by_user,email
object_id,,,,,,,,,,
1,2014-04-22 03:53:30,Clausen August,GUEST_INVITE,1.398139e+09,1,0,11,False,True,yahoo.com
2,2013-11-15 03:45:04,Poole Matthew,ORG_INVITE,1.396238e+09,0,0,1,True,True,gustr.com
3,2013-03-19 23:14:52,Bottrill Mitchell,ORG_INVITE,1.363735e+09,0,0,94,False,True,gustr.com
4,2013-05-21 08:09:28,Clausen Nicklas,GUEST_INVITE,1.369210e+09,0,0,1,False,True,yahoo.com
5,2013-01-17 10:14:20,Raw Grace,GUEST_INVITE,1.358850e+09,0,0,193,False,True,yahoo.com


Data Preparation for Machine Learning

Next, I will prepare the data for use in building a predictive model. I will begin by creating a new df_ml dataframe with our label, the 'adopted user' feature, as the first column, and the 'user_id' as the index:

In [18]:
df_ml = pd.DataFrame({'adopted_user': df_users['adopted'].values}, index=df_users.index)
df_ml

,adopted_user
object_id,
1,False
2,True
3,False
4,False
5,False
...,...
11996,False
11997,False
11998,False


Binary Features

There are a number of binary categorical variables that simply need to be added to the dataframe in their current format:

In [19]:
df_ml['mailing_list'] = df_users['opted_in_to_mailing_list']
df_ml['marketing_drip'] = df_users['enabled_for_marketing_drip']
df_ml['invited_by_user'] = df_users['invited_by_user']
df_ml

,adopted_user,mailing_list,marketing_drip,invited_by_user
object_id,,,,
1,False,1,0,True
2,True,0,0,True
3,False,0,0,True
4,False,0,0,True
5,False,0,0,True
...,...,...,...,...
11996,False,0,0,True
11997,False,0,0,False
11998,False,1,1,True


Categorical Variables

The dataset also has a number of categorical variables. I will convert these into dummy variables and add them to the df_ml dataframe:

In [20]:
# convert org_id column from int to string
df_users['org_id'] = df_users['org_id'].astype('str')

# create the dummy variables
df_dummies = pd.get_dummies(df_users[['creation_source', 'org_id', 'email']])

# add dummy variables to df_model dataframe
df_ml = pd.concat([df_ml, df_dummies], axis=1)
df_ml.head(3)

,adopted_user,mailing_list,marketing_drip,invited_by_user,creation_source_GUEST_INVITE,creation_source_ORG_INVITE,creation_source_PERSONAL_PROJECTS,creation_source_SIGNUP,creation_source_SIGNUP_GOOGLE_AUTH,org_id_0,org_id_1,org_id_10,org_id_100,org_id_101,org_id_102,org_id_103,org_id_104,org_id_105,org_id_106,org_id_107,org_id_108,org_id_109,org_id_11,org_id_110,org_id_111,org_id_112,org_id_113,org_id_114,org_id_115,org_id_116,org_id_117,org_id_118,org_id_119,org_id_12,org_id_120,org_id_121,org_id_122,org_id_123,org_id_124,org_id_125,org_id_126,org_id_127,org_id_128,org_id_129,org_id_13,org_id_130,org_id_131,org_id_132,org_id_133,org_id_134,org_id_135,org_id_136,org_id_137,org_id_138,org_id_139,org_id_14,org_id_140,org_id_141,org_id_142,org_id_143,org_id_144,org_id_145,org_id_146,org_id_147,org_id_148,org_id_149,org_id_15,org_id_150,org_id_151,org_id_152,org_id_153,org_id_154,org_id_155,org_id_156,org_id_157,org_id_158,org_id_159,org_id_16,org_id_160,org_id_161,org_id_162,org_id_163,org_id_164,org_id_165,org_id_166,org_id_167,org_id_168,org_id_169,org_id_17,org_id_170,org_id_171,org_id_172,org_id_173,org_id_174,org_id_175,org_id_176,org_id_177,org_id_178,org_id_179,org_id_18,org_id_180,org_id_181,org_id_182,org_id_183,org_id_184,org_id_185,org_id_186,org_id_187,org_id_188,org_id_189,org_id_19,org_id_190,org_id_191,org_id_192,org_id_193,org_id_194,org_id_195,org_id_196,org_id_197,org_id_198,org_id_199,org_id_2,org_id_20,org_id_200,org_id_201,org_id_202,org_id_203,org_id_204,org_id_205,org_id_206,org_id_207,org_id_208,org_id_209,org_id_21,org_id_210,org_id_211,org_id_212,org_id_213,org_id_214,org_id_215,org_id_216,org_id_217,org_id_218,org_id_219,org_id_22,org_id_220,org_id_221,org_id_222,org_id_223,org_id_224,org_id_225,org_id_226,org_id_227,org_id_228,org_id_229,org_id_23,org_id_230,org_id_231,org_id_232,org_id_233,org_id_234,org_id_235,org_id_236,org_id_237,org_id_238,org_id_239,org_id_24,org_id_240,org_id_241,org_id_242,org_id_243,org_id_244,org_id_245,org_id_246,org_id_247,org_id_248,org_id_249,org_id_25,org_id_250,org_id_251,org_id_252,org_id_253,org_id_254,org_id_255,org_id_256,org_id_257,org_id_258,org_id_259,org_id_26,org_id_260,org_id_261,org_id_262,org_id_263,org_id_264,org_id_265,org_id_266,org_id_267,org_id_268,org_id_269,org_id_27,org_id_270,org_id_271,org_id_272,org_id_273,org_id_274,org_id_275,org_id_276,org_id_277,org_id_278,org_id_279,org_id_28,org_id_280,org_id_281,org_id_282,org_id_283,org_id_284,org_id_285,org_id_286,org_id_287,org_id_288,org_id_289,org_id_29,org_id_290,org_id_291,org_id_292,org_id_293,org_id_294,org_id_295,org_id_296,org_id_297,org_id_298,org_id_299,org_id_3,org_id_30,org_id_300,org_id_301,org_id_302,org_id_303,org_id_304,org_id_305,org_id_306,org_id_307,org_id_308,org_id_309,org_id_31,org_id_310,org_id_311,org_id_312,org_id_313,org_id_314,org_id_315,org_id_316,org_id_317,org_id_318,org_id_319,org_id_32,org_id_320,org_id_321,org_id_322,org_id_323,org_id_324,org_id_325,org_id_326,org_id_327,org_id_328,org_id_329,org_id_33,org_id_330,org_id_331,org_id_332,org_id_333,org_id_334,org_id_335,org_id_336,org_id_337,org_id_338,org_id_339,org_id_34,org_id_340,org_id_341,org_id_342,org_id_343,org_id_344,org_id_345,org_id_346,org_id_347,org_id_348,org_id_349,org_id_35,org_id_350,org_id_351,org_id_352,org_id_353,org_id_354,org_id_355,org_id_356,org_id_357,org_id_358,org_id_359,org_id_36,org_id_360,org_id_361,org_id_362,org_id_363,org_id_364,org_id_365,org_id_366,org_id_367,org_id_368,org_id_369,org_id_37,org_id_370,org_id_371,org_id_372,org_id_373,org_id_374,org_id_375,org_id_376,org_id_377,org_id_378,org_id_379,org_id_38,org_id_380,org_id_381,org_id_382,org_id_383,org_id_384,org_id_385,org_id_386,org_id_387,org_id_388,org_id_389,org_id_39,org_id_390,org_id_391,org_id_392,org_id_393,org_id_394,org_id_395,org_id_396,org_id_397,org_id_398,org_id_399,org_id_4,org_id_40,org_id_400,org_id_401,org_id_402,org_id_403,org_id_404,org_id_405,org_id_406,org_id_407,org_id_408,org_id_409,org_id_41,org_id_410,org_id_4

Predictive Modeling

Next, I will train a predictive model. Referening the scikit-learn model selection flowchart recommends trying an ensemble model for classification problems with less than 100k samples. I will try Random Forest as my ensemble classifier.

First, I will split the data into training and test sets:

In [21]:
# divide the data into label and features for use in ml models
X = df_ml.iloc[:, 1:]
y = df_ml.loc[:, 'adopted_user']

# split the data into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=6, stratify=y)

Random Forest Classifier

In [22]:
# train the model and make predictions on the training and test sets
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(class_weight='balanced_subsample', random_state=6)
rfc.fit(X_train, y_train)
y_pred_rfc = rfc.predict(X_test)
print('Accuracy on training set = {}'.format(rfc.score(X_train, y_train)))
print('Accuracy on test set = {}'.format(rfc.score(X_test, y_test)))

Accuracy on training set = 0.9427777777777778
Accuracy on test set = 0.837


In [23]:
# generate classification report of model performance
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_rfc))

              precision    recall  f1-score   support

       False       0.88      0.94      0.91      2639
        True       0.18      0.10      0.13       361

    accuracy                           0.84      3000
   macro avg       0.53      0.52      0.52      3000
weighted avg       0.80      0.84      0.82      3000



In [24]:
# generate confusion matrix for model performance
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred_rfc))

[[2476  163]
 [ 326   35]]


In [25]:
# create dataframe of feature importances from model
df_features = pd.DataFrame({'rfc': rfc.feature_importances_}, index=df_ml.columns[1:])

# sort by highest values
df_features.sort_values('rfc', ascending=False)[:15]

,rfc
mailing_list,0.036834
marketing_drip,0.025103
email_gmail.com,0.016393
email_yahoo.com,0.013526
email_gustr.com,0.013521
email_jourrapide.com,0.012945
email_other,0.012826
creation_source_PERSONAL_PROJECTS,0.011557
email_cuvox.de,0.010256
email_hotmail.com,0.009893


While at first blush it may appear that the "out-of-the-box" Random Forest Classifier performed well by classifying 84% of the test set samples correctly, we must keep in mind that the null accuracy rate is 88% (if the model simply predicted that every user did not adopt the product). When data is imbalanced to the degree that we have here, it is more important to analyze how well the model was able to identify the minority class (in this case, the adopted users) as models tend to favor majority classes when data is imbalanced. Focusing on identifying the minority class of adopted users is particularly important here given that the goal of this project is to identify which of the provided features is most predictive of future user product adoption.

Here, the classification report and confusion matrix demonstrates that the "out-of-the-box" model correctly identified adopted users at an alarmingly low rate (only 29 of 361 in the test set). The feature importance values of the model therefore are not going to provide much insight as to which of the features are most predictive of future user product adoption.

All is not lost, however. The Random Forest Classifier has a number of hyperparameters that can be tuned to improve performance. Let's tune some of them to see if it improves the model's accuracy rate on the adopted users:

In [26]:
# tune some of the hyperparameters with gridsearch
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
rfc = RandomForestClassifier(random_state=6)
param_grid = {'n_estimators': [100, 300, 500], 'max_depth': [5, 15, 30], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 3, 5]}
grid = GridSearchCV(rfc, param_grid, cv=3, scoring='accuracy')
grid.fit(X_train, y_train)
y_pred_rfc = grid.predict(X_test)
print(grid.score(X_test, y_test))

0.8796666666666667


In [27]:
grid.best_score_

0.8795555555555555

In [28]:
grid.best_estimator_

RandomForestClassifier(max_depth=5, random_state=6)


It looks like tuning some of the hyperparameters improved the model's overall accuracy. Let's use the classifier with the tuned hyperparameter values on the training and test sets and see if that enables the classifier to identify more adopted users accurately:

Tuned Model Performance

In [29]:
# train the model and make predictions on the training and test sets
from sklearn.ensemble import RandomForestClassifier
rfc_tuned = RandomForestClassifier(n_estimators=100, max_depth=5, min_samples_split=2, min_samples_leaf=1, criterion='gini', class_weight='balanced_subsample', random_state=6)
rfc_tuned.fit(X_train, y_train)
y_pred_rfc_tuned = rfc_tuned.predict(X_test)
print('Accuracy on training set = {}'.format(rfc_tuned.score(X_train, y_train)))
print('Accuracy on test set = {}'.format(rfc_tuned.score(X_test, y_test)))

Accuracy on training set = 0.6206666666666667
Accuracy on test set = 0.6


In [30]:
# generate classification report of model performance
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_rfc_tuned))

              precision    recall  f1-score   support

       False       0.90      0.61      0.73      2639
        True       0.15      0.51      0.23       361

    accuracy                           0.60      3000
   macro avg       0.53      0.56      0.48      3000
weighted avg       0.81      0.60      0.67      3000



In [31]:
# generate confusion matrix for model performance
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred_rfc_tuned))

[[1616 1023]
 [ 177  184]]


While the overall accuracy of the tuned classifier decreased significantly, the confusion matrix demonstrates that the model performed significantly better in identifying over half of the adopted users accurately (183 of 361 in the test set). We therefore should be able to gain insight as to which particular features are most predictive of future user adoption by seeing which of the features received the highest feature importance increase by the tuned model. Let's take a look:

In [32]:
# add the tuned classifier feature importance values to the df_features dataframe
df_features['rfc_tuned'] = rfc_tuned.feature_importances_

# add column calculating the difference between the feature importance values
df_features['difference'] = df_features['rfc_tuned'] - df_features['rfc']

# sort by highest difference
df_features.sort_values('difference', ascending=False)[:15]

,rfc,rfc_tuned,difference
creation_source_PERSONAL_PROJECTS,0.011557,0.080421,0.068864
org_id_0,0.008809,0.065023,0.056215
creation_source_GUEST_INVITE,0.008469,0.063778,0.055309
email_yahoo.com,0.013526,0.044497,0.030971
email_hotmail.com,0.009893,0.037771,0.027877
creation_source_SIGNUP_GOOGLE_AUTH,0.005455,0.022810,0.017355
email_gmail.com,0.016393,0.033124,0.016731
org_id_1,0.007304,0.022089,0.014784
org_id_3,0.006190,0.017236,0.011046
creation_source_ORG_INVITE,0.009336,0.019857,0.010521


Final Analysis/Conclusions

The above table demonstrates that the factors that most likely lead to a particular user adopting the product are when the user is invited to join another user's workspace or is invited to an organization as a guest. It also demonstrates that being a member of certain organizations, and in particular 'org_id_0,' makes it more likely for a user to adopt the product. The tuned classifier also found three of the email domains to be of increased importance (and signing up using google authentication), as well as being invited to become a full member of an organization.

Based on the above, I would make the following recommendations to Relax regarding their attempts to improve future user adoption:

Run promotions seeking to incentivize users to invite prospective users to join their personal workspaces on the product;

Run promotions seeking to incentivize organizations to grow by sending guest and/or full invites to prospective users to join the product;

Promote certain organizations identified by the model above (in particular org_id_0) to all users; and

Target its regular email marketing drip to existing and prospective users with yahoo.com, hotmail.com, and gmail.com email addresses.